In [1]:
from __future__ import division, print_function, absolute_import
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, Model
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50


In [2]:
cd F:\CZR\ANU courses\2019 S1\ENGN4528\project\tensorflow_cnn

F:\CZR\ANU courses\2019 S1\ENGN4528\project\tensorflow_cnn


In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [4]:
# # 图片路径，两组标签都在该目录下
# cwd = r".\resize224"

# # tfrecord文件保存路径
# file_path = r"./"

# # 每个tfrecord存放图片个数
# bestnum = 10000

# # 第几个图片
# num = 0

# # 第几个TFRecord文件
# recordfilenum = 0

# # 将labels放入到classes中
# classes = [str(i) for i in range(37)]
# print(classes)

# # tfrecords格式文件名
# ftrecordfilename = ("traindata_cat&dog_augmented.tfrecords-%.3d" % recordfilenum)
# writer = tf.python_io.TFRecordWriter(os.path.join(file_path, ftrecordfilename))


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36']


In [5]:
# for index, name in enumerate(classes):

#     class_path = os.path.join(cwd, name)
#     for img_name in os.listdir(class_path):
#         num = num + 1
#         if num > bestnum:  # 超过10000，写入下一个tfrecord
#             num = 1
#             recordfilenum += 1
#             ftrecordfilename = ("traindata_cat&dog_augmented.tfrecords-%.3d" % recordfilenum)
#             writer = tf.python_io.TFRecordWriter(os.path.join(file_path, ftrecordfilename))
#         img_path = os.path.join(class_path, img_name)  # 每一个图片的地址
#         img = Image.open(img_path, 'r')
#         img_raw = img.tobytes()  # 将图片转化为二进制格式
#         example = tf.train.Example(
#             features=tf.train.Features(feature={
#                 'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
#                 'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
#             }))
#         writer.write(example.SerializeToString())  # 序列化为字符串
# writer.close()

In [6]:
def read_and_decode_tfrecord(filename):
    filename_deque = tf.train.string_input_producer(filename)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_deque)
    features = tf.parse_single_example(serialized_example, features={
        'label': tf.FixedLenFeature([], tf.int64),
        'img_raw': tf.FixedLenFeature([], tf.string)})
    label = tf.cast(features['label'], tf.int32)
    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [224, 224, 3])
    img = tf.cast(img, tf.float32) / 255.0      #将矩阵归一化0-1之间
    return img, label

In [7]:
os.listdir()

['0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'cats only',
 'cats tfrecord',
 'dogs only',
 'dogs tfrecord',
 'resize224',
 'resize224.zip',
 'resize224_augmented.zip',
 'resnet50.hdf5',
 'test224',
 'test224.zip',
 'train224',
 'train224.zip',
 'traindata_cat&dog_augmented.tfrecords-000',
 'traindata_cat&dog_augmented.tfrecords-001',
 'traindata_cat&dog_augmented.tfrecords-002',
 'traindata_cat&dog_augmented.tfrecords-003',
 'traindata_cat&dog_augmented.tfrecords-004',
 'traindata_cat&dog_no_augmented.tfrecords-000',
 'traindata_cat&dog_no_augmented.tfrecords-001',
 'traindata_cat&dog_no_augmented.tfrecords-002',
 'traindata_cat&dog_no_augmented.tfrecords-003',
 'traindata_cat&dog_no_augmented.tfrecords-004',
 'traindata_cat&dog_no_augmented.tfrecords-005',
 'traindata_cat&dog_no_augm

In [8]:
train_list = ['traindata_cat&dog_augmented.tfrecords-000',
 'traindata_cat&dog_augmented.tfrecords-001',
 'traindata_cat&dog_augmented.tfrecords-002',
 'traindata_cat&dog_augmented.tfrecords-003',
 'traindata_cat&dog_augmented.tfrecords-004']

In [9]:
# Training Parameters
learning_rate = 0.0001
num_steps = 1
batch_size = 32
display_step = 10

# Network Parameters
num_input = 50176
num_classes = 37

# tf Graph input
X = tf.placeholder(tf.float32, [None, 224, 224, 3])
Y = tf.placeholder(tf.float32, [None])
y_ = tf.placeholder(tf.float32, [None, num_classes])

In [10]:
img, label = read_and_decode_tfrecord(train_list)
img_batch, label_batch = tf.train.shuffle_batch([img, label], num_threads=32, batch_size=batch_size, capacity=10000,min_after_dequeue=9900)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecor

In [11]:
# 将label值进行onehot编码
one_hot_labels = tf.one_hot(indices=tf.cast(Y, tf.int32), depth=37)


In [12]:
input = layers.Input((224, 224, 3))
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model(input)
x = layers.Flatten()(x)
prediction = layers.Dense(num_classes,activation='softmax')(x)
resnet50 = Model(input, prediction)
resnet50.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss=tf.keras.losses.categorical_crossentropy, metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.


C:\ProgramData\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [13]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [14]:
# Start training
saver = tf.train.Saver(max_to_keep=1)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
counter=0
# Run the initializer
for epoch in range(1, 16):
    print("epoch:", epoch)
    with tf.device('/gpu:0'):
        for batch in range(1,1475): #47208 pics /7078
            batch_x, batch_y = sess.run([img_batch, label_batch])
            batch_y = sess.run(one_hot_labels, feed_dict={Y: batch_y})
            loss, acc = resnet50.train_on_batch(batch_x, batch_y)
            print("Epoch " + str(epoch) + ", Minibatch Loss= " + \
                        "{:.4f}".format(loss) + ", Training Accuracy= " + \
                        "{:.3f}".format(acc))
        resnet50.save_weights('resnet50.hdf5')
print("Optimization Finished!")
coord.request_stop()
coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
epoch: 1
Instructions for updating:
Use tf.cast instead.
Epoch 1, Minibatch Loss= 5.5518, Training Accuracy= 0.062
Epoch 1, Minibatch Loss= 3.5129, Training Accuracy= 0.188
Epoch 1, Minibatch Loss= 2.5289, Training Accuracy= 0.344
Epoch 1, Minibatch Loss= 1.9966, Training Accuracy= 0.406
Epoch 1, Minibatch Loss= 2.1066, Training Accuracy= 0.406
Epoch 1, Minibatch Loss= 1.5252, Training Accuracy= 0.469
Epoch 1, Minibatch Loss= 1.6954, Training Accuracy= 0.594
Epoch 1, Minibatch Loss= 1.5886, Training Accuracy= 0.500
Epoch 1, Minibatch Loss= 3.4437, Training Accuracy= 0.594
Epoch 1, Minibatch Loss= 1.2545, Training Accuracy= 0.625
Epoch 1, Minibatch Loss= 1.7647, Training Accuracy= 0.656
Epoch 1, Minibatch Loss= 2.1642, Training Accuracy= 0.625
Epoch 1, Minibatch Loss= 1.5971, Training Accuracy= 0.562
Epoch 1, Minibatch Loss= 1.7641, Training Accuracy= 0.562
Epoch 1, Minibatch Loss= 1.5600, Training Accura

Epoch 1, Minibatch Loss= 6.2800, Training Accuracy= 0.562
Epoch 1, Minibatch Loss= 4.4489, Training Accuracy= 0.688
Epoch 1, Minibatch Loss= 6.2926, Training Accuracy= 0.500
Epoch 1, Minibatch Loss= 5.5407, Training Accuracy= 0.656
Epoch 1, Minibatch Loss= 6.7319, Training Accuracy= 0.469
Epoch 1, Minibatch Loss= 4.6627, Training Accuracy= 0.656
Epoch 1, Minibatch Loss= 7.0741, Training Accuracy= 0.562
Epoch 1, Minibatch Loss= 4.6119, Training Accuracy= 0.656
Epoch 1, Minibatch Loss= 4.6577, Training Accuracy= 0.688
Epoch 1, Minibatch Loss= 5.7762, Training Accuracy= 0.625
Epoch 1, Minibatch Loss= 3.9071, Training Accuracy= 0.656
Epoch 1, Minibatch Loss= 5.5555, Training Accuracy= 0.656
Epoch 1, Minibatch Loss= 5.0619, Training Accuracy= 0.688
Epoch 1, Minibatch Loss= 6.2427, Training Accuracy= 0.562
Epoch 1, Minibatch Loss= 7.0119, Training Accuracy= 0.500
Epoch 1, Minibatch Loss= 7.1131, Training Accuracy= 0.531
Epoch 1, Minibatch Loss= 6.4653, Training Accuracy= 0.562
Epoch 1, Minib

Epoch 1, Minibatch Loss= 0.5239, Training Accuracy= 0.812
Epoch 1, Minibatch Loss= 0.4874, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.3895, Training Accuracy= 0.875
Epoch 1, Minibatch Loss= 0.6317, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.4311, Training Accuracy= 0.906
Epoch 1, Minibatch Loss= 0.3378, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.2635, Training Accuracy= 0.938
Epoch 1, Minibatch Loss= 0.7967, Training Accuracy= 0.750
Epoch 1, Minibatch Loss= 0.2950, Training Accuracy= 0.906
Epoch 1, Minibatch Loss= 0.8558, Training Accuracy= 0.812
Epoch 1, Minibatch Loss= 0.1456, Training Accuracy= 0.938
Epoch 1, Minibatch Loss= 0.4546, Training Accuracy= 0.875
Epoch 1, Minibatch Loss= 0.2587, Training Accuracy= 0.969
Epoch 1, Minibatch Loss= 0.4412, Training Accuracy= 0.812
Epoch 1, Minibatch Loss= 0.3905, Training Accuracy= 0.812
Epoch 1, Minibatch Loss= 0.4185, Training Accuracy= 0.938
Epoch 1, Minibatch Loss= 0.5386, Training Accuracy= 0.875
Epoch 1, Minib

Epoch 1, Minibatch Loss= 0.3677, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.3188, Training Accuracy= 0.906
Epoch 1, Minibatch Loss= 1.1843, Training Accuracy= 0.656
Epoch 1, Minibatch Loss= 0.4843, Training Accuracy= 0.781
Epoch 1, Minibatch Loss= 0.4100, Training Accuracy= 0.875
Epoch 1, Minibatch Loss= 0.5115, Training Accuracy= 0.812
Epoch 1, Minibatch Loss= 0.5125, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.6101, Training Accuracy= 0.781
Epoch 1, Minibatch Loss= 0.8635, Training Accuracy= 0.781
Epoch 1, Minibatch Loss= 0.7364, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.2287, Training Accuracy= 0.938
Epoch 1, Minibatch Loss= 0.4690, Training Accuracy= 0.875
Epoch 1, Minibatch Loss= 0.3360, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.3503, Training Accuracy= 0.938
Epoch 1, Minibatch Loss= 0.9492, Training Accuracy= 0.719
Epoch 1, Minibatch Loss= 0.5399, Training Accuracy= 0.812
Epoch 1, Minibatch Loss= 0.8769, Training Accuracy= 0.781
Epoch 1, Minib

Epoch 1, Minibatch Loss= 1.0879, Training Accuracy= 0.719
Epoch 1, Minibatch Loss= 0.2499, Training Accuracy= 0.875
Epoch 1, Minibatch Loss= 0.2323, Training Accuracy= 0.938
Epoch 1, Minibatch Loss= 0.1824, Training Accuracy= 0.969
Epoch 1, Minibatch Loss= 0.9113, Training Accuracy= 0.812
Epoch 1, Minibatch Loss= 1.4975, Training Accuracy= 0.750
Epoch 1, Minibatch Loss= 0.5921, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.6633, Training Accuracy= 0.750
Epoch 1, Minibatch Loss= 0.7354, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.1769, Training Accuracy= 0.938
Epoch 1, Minibatch Loss= 0.1096, Training Accuracy= 0.938
Epoch 1, Minibatch Loss= 0.7456, Training Accuracy= 0.781
Epoch 1, Minibatch Loss= 0.7856, Training Accuracy= 0.844
Epoch 1, Minibatch Loss= 0.4929, Training Accuracy= 0.875
Epoch 1, Minibatch Loss= 1.3157, Training Accuracy= 0.656
Epoch 1, Minibatch Loss= 0.7829, Training Accuracy= 0.781
Epoch 1, Minibatch Loss= 0.4284, Training Accuracy= 0.812
Epoch 1, Minib

...

Epoch 15, Minibatch Loss= 0.0798, Training Accuracy= 0.938
Epoch 15, Minibatch Loss= 0.1134, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.1503, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.1370, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0022, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0009, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0007, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0045, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0445, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0008, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0001, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0081, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0001, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0178, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0003, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0004, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0093, Training Accuracy= 1.0

Epoch 15, Minibatch Loss= 0.0002, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0002, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0034, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0964, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0038, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0080, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0097, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0798, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0004, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0770, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0029, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0990, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0175, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0068, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0073, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0015, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0122, Training Accuracy= 1.0

Epoch 15, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0001, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0012, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0035, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0153, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0017, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0030, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.1409, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.1549, Training Accuracy= 0.938
Epoch 15, Minibatch Loss= 0.0005, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0661, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0043, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0119, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0124, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0036, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0001, Training Accuracy= 1.0

Epoch 15, Minibatch Loss= 0.0027, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0002, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0011, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0081, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0004, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0173, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0004, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0012, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0002, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0091, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0075, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0006, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0011, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0002, Training Accuracy= 1.0

Epoch 15, Minibatch Loss= 0.0024, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0048, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.2250, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0881, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0828, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0438, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0514, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0124, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0211, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0212, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.1158, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0708, Training Accuracy= 0.969
Epoch 15, Minibatch Loss= 0.0146, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.2835, Training Accuracy= 0.938
Epoch 15, Minibatch Loss= 0.0003, Training Accuracy= 1.000
Epoch 15, Minibatch Loss= 0.0620, Training Accuracy= 0.969
Optimization Finished!


In [15]:
counter=0
right=0
for label in os.listdir (r'.\test224'):


    test_dir= os.path.join(r'.\test224', label)

    for pic in os.listdir(test_dir):
        img_path = os.path.join(test_dir, pic)
        img = Image.open(img_path)
        img = img.resize((224, 224))
        img = tf.reshape(img, [1, 224, 224, 3])
        img1 = tf.reshape(img, [1, 224, 224, 3])
        img = tf.cast(img, tf.float32) / 255.0
        b_image, b_image_raw = sess.run([img, img1])
        haha_sm = resnet50.predict(b_image)
        print(sess.run(tf.argmax(haha_sm, 1)),int(label))
        if sess.run(tf.argmax(haha_sm, 1))== int(label):
            right +=1

        counter +=1
acc=right/counter
print(acc)

[21] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[27] 0
[0] 0
[0] 0
[1] 1
[1] 1
[1] 1
[28] 1
[34] 1
[1] 1
[1] 1
[1] 1
[1] 1
[8] 10
[1] 10
[10] 10
[8] 10
[10] 10
[21] 10
[1] 10
[11] 11
[11] 11
[11] 11
[11] 11
[11] 11
[11] 11
[11] 11
[11] 11
[12] 12
[22] 12
[13] 12
[30] 12
[12] 12
[12] 12
[12] 12
[22] 12
[13] 13
[28] 13
[17] 13
[3] 13
[12] 13
[13] 13
[13] 13
[4] 13
[14] 14
[8] 14
[14] 14
[30] 14
[14] 14
[14] 14
[14] 14
[14] 14
[15] 15
[15] 15
[1] 15
[15] 15
[29] 15
[15] 15
[28] 15
[15] 15
[13] 16
[16] 16
[16] 16
[16] 16
[16] 16
[16] 16
[16] 16
[16] 16
[17] 17
[17] 17
[17] 17
[17] 17
[17] 17
[17] 17
[17] 17
[17] 17
[18] 18
[18] 18
[18] 18
[18] 18
[18] 18
[18] 18
[18] 18
[18] 18
[19] 19
[19] 19
[19] 19
[8] 19
[19] 19
[19] 19
[8] 19
[22] 19
[2] 2
[1] 2
[34] 2
[1] 2
[8] 2
[2] 2
[32] 2
[2] 2
[5] 20
[20] 20
[10] 20
[23] 20
[32] 20
[20] 20
[11] 20
[20] 20
[21] 21
[21] 21
[21] 21
[21] 21
[10] 21
[21] 21
[21] 21
[21] 21
[22] 22
[22] 22
[19] 22
[22] 22
[22] 22
[22] 22
[22] 22
[22] 22
[23] 23
[23] 23
[23